In [1]:
import os, kagglehub
from huggingface_hub import snapshot_download
import requests

In [2]:
file_url = 'https://github.com/ollama/ollama/releases/download/v0.13.1/ollama-linux-amd64.tgz'
response = requests.get(file_url, stream=True)

In [3]:
if response.status_code == 200:
    print('File downloaded?')
else:
    print(f"Failed to download file. Status code: {response.status_code}")

File downloaded?


In [4]:
local_filename = "ollama-linux-amd64.tgz" # Desired local filename
with open(local_filename, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)
print(f"File downloaded successfully to {local_filename}")

File downloaded successfully to ollama-linux-amd64.tgz


In [5]:
os.makedirs('../model', exist_ok=True)
local_dir = '../model'
snapshot_download(
    repo_id="unsloth/gpt-oss-120b-GGUF",
    local_dir=local_dir,
    force_download=True,
    allow_patterns=[
        '.gitattributes',
        'README.md',
        'config.json',
        'params',
        'template',
        'UD-Q4_K_XL/gpt-oss-120b-UD-Q4_K_XL-00002-of-00002.gguf',
        'UD-Q4_K_XL/gpt-oss-120b-UD-Q4_K_XL-00001-of-00002.gguf',
    ],
)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

'/workspace/download-models-from-hf-and-upload-to-kaggle/model'

In [6]:
import os, subprocess, threading, time

In [7]:
OLLAMA_ARCHIVE_PATH = "../ollama-linux-amd64.tgz"
MODEL_GGUF_PATH = "../model"

In [8]:
# ==========================================
# 3. Ollama 압축 해제 및 세팅 (여기가 중요!)
# ==========================================
print("Extracting Ollama...")

# 1) 압축 풀 폴더 만들기
subprocess.run("mkdir -p /tmp/ollama_pkg", shell=True)

# 2) 압축 해제 (tar -xzf)
# -C 옵션: /tmp/ollama_pkg 폴더 안에다가 풀어라
subprocess.run(f"tar -xzf {OLLAMA_ARCHIVE_PATH} -C /tmp/ollama_pkg", shell=True)

# 3) 진짜 실행 파일 위치 지정
# 압축 풀면 보통 bin/ollama에 실행 파일이 들어있음
OLLAMA_BIN = "/tmp/ollama_pkg/bin/ollama"

# 실행 권한 주기
subprocess.run(f"chmod +x {OLLAMA_BIN}", shell=True)

Extracting Ollama...


tar (child): ../ollama-linux-amd64.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
chmod: cannot access '/tmp/ollama_pkg/bin/ollama': No such file or directory


CompletedProcess(args='chmod +x /tmp/ollama_pkg/bin/ollama', returncode=1)

In [9]:
# ==========================================
# 4. 서버 실행
# ==========================================
def start_ollama_server():
    env = os.environ.copy()
    env["HOME"] = "/tmp"
    
    # [중요] 압축 푼 폴더 안의 lib 폴더를 라이브러리 경로에 추가해야 GPU를 인식함
    env["LD_LIBRARY_PATH"] = f"/tmp/ollama_pkg/lib:{env.get('LD_LIBRARY_PATH', '')}"
    
    with open("/tmp/ollama_log.txt", "w") as log_file:
        # 압축 푼 바이너리(OLLAMA_BIN)를 실행
        subprocess.Popen([OLLAMA_BIN, "serve"], env=env, stdout=log_file, stderr=log_file)

thread = threading.Thread(target=start_ollama_server)
thread.start()

print("Waiting for Ollama server to start...")
time.sleep(10) # 압축 풀고 실행하는 거라 시간을 좀 넉넉히 줌 (5초 -> 10초)

print("Ollama Server Started!")

Waiting for Ollama server to start...
Ollama Server Started!


In [11]:
folder_path = "../model/UD-Q4_K_XL"
abs_folder_path = os.path.abspath(folder_path)
abs_folder_path

'/workspace/download-models-from-hf-and-upload-to-kaggle/model/UD-Q4_K_XL'

In [12]:
files = [f for f in os.listdir(abs_folder_path) if f.endswith('.gguf')]

for i, f in enumerate(files):
    print(f"[{i}] {f}")

[0] gpt-oss-120b-UD-Q4_K_XL-00002-of-00002.gguf
[1] gpt-oss-120b-UD-Q4_K_XL-00001-of-00002.gguf


In [14]:
# 진짜 실행 파일 경로 (압축 풀린 곳)
OLLAMA_BIN = "/tmp/ollama_pkg/bin/ollama"
MODEL_GGUF_PATH = os.path.join(abs_folder_path, "gpt-oss-120b-UD-Q4_K_XL-00001-of-00002.gguf")

# Modelfile 생성
modelfile_content = f"FROM {MODEL_GGUF_PATH}\n"
with open("/tmp/Modelfile", "w") as f:
    f.write(modelfile_content)

# [핵심 수정] /tmp/ollama 대신 {OLLAMA_BIN} 변수를 사용해야 함!
# HOME=/tmp 환경변수도 같이 넣어줘야 권한 문제 안 생김
create_command = f"HOME=/tmp {OLLAMA_BIN} create my-model -f /tmp/Modelfile"

subprocess.run(create_command, shell=True)

print("Model Created Successfully!")

gathering model components ⠙ gathering model components ⠙ gathering model components ⠸ gathering model components ⠸ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠏ gathering model components ⠋ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠇ gathering model components ⠋ gathering model components ⠋ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠧ gathering model components ⠏ gathering model components ⠋ gathering model components ⠋ gathering model components ⠙ gathering model components ⠸ gathering model components ⠼ gathering mode

Model Created Successfully!


gathering model components 
copying file sha256:c7b59d68c889ada280736d9b82685baeb6152f4d7423a169a369bb3492b3056e 100% 
parsing GGUF ⠇ gathering model components 
copying file sha256:c7b59d68c889ada280736d9b82685baeb6152f4d7423a169a369bb3492b3056e 100% 
parsing GGUF 
using existing layer sha256:c7b59d68c889ada280736d9b82685baeb6152f4d7423a169a369bb3492b3056e 
writing manifest 
success 


In [17]:
cache_folder_path = "../model_cache"
os.makedirs(cache_folder_path, exist_ok=True)
abs_folder_path = os.path.abspath(cache_folder_path)
abs_folder_path

'/workspace/download-models-from-hf-and-upload-to-kaggle/model_cache'

In [18]:
print("Archiving Ollama Model Cache...")

# 1. Ollama가 생성한 데이터 폴더 (.ollama)를 통째로 압축
# 우리는 아까 HOME=/tmp로 설정했으니, 데이터는 /tmp/.ollama 에 있습니다.
# 압축 파일은 /kaggle/working (Output)에 저장됩니다.
subprocess.run(f"tar -czf {abs_folder_path}/ollama_model_cache.tar.gz -C /tmp .ollama", shell=True)

print(f"Archive created at {abs_folder_path}/ollama_model_cache.tar.gz")
print("이제 우측 패널 Output에서 이 파일을 다운로드 받아서, 새 데이터셋으로 올리세요!")

Archiving Ollama Model Cache...


Archive created at /workspace/download-models-from-hf-and-upload-to-kaggle/model_cache/ollama_model_cache.tar.gz
이제 우측 패널 Output에서 이 파일을 다운로드 받아서, 새 데이터셋으로 올리세요!


In [19]:
kagglehub.login()

In [20]:
handle = 'max6296/unsloth-gpt-oss-120b-4bit-ollama-cache/GGUF/default'

In [21]:
kagglehub.model_upload(handle, cache_folder_path)

Uploading Model https://www.kaggle.com/models/max6296/unsloth-gpt-oss-120b-4bit-ollama-cache/GGUF/default ...
Model 'unsloth-gpt-oss-120b-4bit-ollama-cache' does not exist or access is forbidden for user 'max6296'. Creating or handling Model...
Model 'unsloth-gpt-oss-120b-4bit-ollama-cache' Created.
Starting upload for file ../model_cache/ollama_model_cache.tar.gz


Uploading: 100%|██████████| 46.4G/46.4G [04:05<00:00, 189MB/s] 

Upload successful: ../model_cache/ollama_model_cache.tar.gz (43GB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/max6296/unsloth-gpt-oss-120b-4bit-ollama-cache/GGUF/default
